## Задача (варіант 2)

Задача прийняття рішення полягає в оцінюванні наступних чотирьох варіантів
вкладення коштів: придбання акцій, оформлення депозиту, придбання
облігацій, придбання дорогоцінних металів за критерієм «надійність
вкладення коштів». За результатами парних порівнянь цих варіантів
встановлено, що другий варіант ненабагато кращий за перший і третій
варіанти і суттєво кращий за четвертий, перший варіант має однакову
надійність, що і третій, і ненабагато кращий за четвертий варіант,
перевага третього варіанту над четвертим – між слабкою і суттєвою.

## Матриця парних порівнянь

|Інтенсивність важливості | Якісна оцінка|
|-------------------------|--------------|
| 1 | однаково важливі |
| 3 | ненабагато важливіші |
| 5 | суттєво важливіші |
| 7 | значно важливіші |
| 9 | абсолютно важливіші |
2, 4, 6, 8 - проміжні

In [1]:
import numpy as np

Матриця парних порівнянь $D = \{d_{ij} | i, j = 1, \dots, 4\}$ альтернатив $A = \{a_i\}, i = 1, \dots, 4$.

Для мультиплікативних груп $d_{ij}$ вказує **у скільки** разів альтернатива $a_i$ переважає альтернативу $a_j$ вдносно критерію, за яким будується матриця $D$.

In [2]:
D = np.array([[1.0, 1/3.0, 1.0, 3.0],
              [3.0, 1.0, 3.0, 5.0],
              [1.0, 1/3.0, 1.0, 4.0],
              [1/3.0, 1/5.0, 1/4.0, 1.0]])
D

array([[1.        , 0.33333333, 1.        , 3.        ],
       [3.        , 1.        , 3.        , 5.        ],
       [1.        , 0.33333333, 1.        , 4.        ],
       [0.33333333, 0.2       , 0.25      , 1.        ]])

Перевірка матриці парних порівнянь на коректність(якщо альтернатива $a_i$ краща за альтернативу $a_j$ у $k$ разів, тоді $d_{ij} = k, d_{ji} = \frac{1}{k}$).

In [4]:
def check_inverse_symmetry(matrix):
    for i in range(len(matrix)):
        for j in range(i, len(matrix[i])):
            if matrix[j][i] != 1./matrix[i][j] or matrix[i][j] <= 0:
                return False
    return True

print("Matrix satisfies inverse symmetry property: {}".format(check_inverse_symmetry(D)))

Matrix satisfies inverse symmetry property: True


## Ваги альтернатив

Метою спеціаліста з методу аналізу ієрархій є побудова ваг для кожної з альтернатив: $a_i, i = 1, \dots, 4$. Для цього існують різні методи. Нижче застосовано метод головного власного вектору(eigenvector method, EM)
### Row Geometric Mean Method (RGMM)

Шукаємо усереднені ваги для матриці парних порівнянь $D$ за формулою
$$w_i=\frac{\sqrt[n]{\prod_{j=1}^{n}d_ij}}{\sum_{k=1}^{n}\sqrt[n]{\prod_{j=1}^{n}d_kj}}$$

In [6]:
import math
weighcalc = lambda x: math.pow(np.prod(x),1/4.)
v = [weighcalc(di) for di in D]
w = [vi/np.sum(v) for vi in v]
for i in range(len(w)):
    print('Normalized weight #{}: {} \nRaw weight: {}'.format(i + 1, w[i], v[i]))
    print('-' * 40)
print('Sanity check: {}'.format(np.sum(w)))

Normalized weight #1: 0.19904878138811033 
Raw weight: 1.0
----------------------------------------
Normalized weight #2: 0.5155403375321199 
Raw weight: 2.5900200641113513
----------------------------------------
Normalized weight #3: 0.21389183544578083 
Raw weight: 1.074569931823542
----------------------------------------
Normalized weight #4: 0.07151904563398906 
Raw weight: 0.3593041119630842
----------------------------------------
Sanity check: 1.0000000000000002


Обчислимо міру неузгодженості МПП за RGMM $GCI$

In [7]:
temp = [ [math.log(D[i, j] * v[j] / v[i]) ** 2 for j in range(i, 4)] for i in range (4)]
GCI = np.sum(np.sum(temp)) / 3.

print('GCI = {}'.format(GCI))

GCI = 0.09955830402757827


**МПП допустимо неузгоджена, бо GСІ не перевищує попрогове значення для n = 4: 0.3526**

In [8]:
print("Final weights: {}".format(w))

Final weights: [0.19904878138811033, 0.5155403375321199, 0.21389183544578083, 0.07151904563398906]
